# Computing the Lagrangian Descriptors of Nonlinear Dynamical Systems

## Functions

__NOTE__: For testing, execute all the cells below before executing/modifying all the benchmark examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.integrate import solve_ivp as integrator # RK45 (default)

INPUT PARAMETERS

In [ ]:
###################################
# Integration parameters
t0 = 0
tau = 12
dt = 0.1
###################################
# Lagrangian descriptor parameters
p_norm = 1/2
flag_m = True
###################################
# Mesh parameters
x_min,x_max = [-1.6, 1.6]
y_min,y_max = [-1, 1]
Nx, Ny = [300, 300]

GRID_PARAMETERS = [(x_min, x_max, Nx), (y_min, y_max, Ny)]

dim = 2
shape = (Nx*Ny, dim)
###################################
# Time Dependent Perurbation (for Hamilton Duffing)
flag_pert = 1;
perturbation_params = [0.15, 0.5]

In [ ]:
def generate_points(GRID_PARAMETERS):
    x_min, x_max, Nx = GRID_PARAMETERS[0]
    y_min, y_max, Ny = GRID_PARAMETERS[1]
    points_x = np.linspace(x_min, x_max, Nx)
    points_y = np.linspace(y_min, y_max, Ny)    
    Y, X = np.meshgrid(points_y, points_x)  # Grid in phase-space
    mesh = np.transpose([X.flatten(), Y.flatten()]) # 2D grid
    return mesh

Turn phase-space vector field to high-dimensional vector field

In [ ]:
def vector_field_flat(t, points, vector_field):
    u = points.reshape(shape)
    v = vector_field(t, u)
    return v.flatten()

In [ ]:
def accumulate_lagrangian_descriptor(LD, points_initial, points_now, p_norm = 0.5, flag_m = True):
    if flag_m: # p-norm LDs
        LD_accum = LD + ((abs(points_now[:,0] - points_initial[:,0]))**p_norm + (abs(points_now[:,1] - points_initial[:,1]))**p_norm) * dt ** (1 - p_norm);
    else:  # arclength LDs
        LD_accum = LD + sqrt((points_now[:,0] - points_now[:,0])**2 + (points_now[:,1] - points_initial[:,1])**2);
    return LD_accum

In [ ]:
def lagrangian_descriptor(points_final, p_norm = 0.5, flag_m = True):
    diff_points=points_final[1:]-points_final[:-1]
    if flag_m: # p-norm LDs
        LDrows = np.sum(np.abs(diff_points)**p_norm, axis=2)
        LD = np.sum(LDrows, axis=0)
    else:  # arclength LDs
        LDrows = np.sqrt(np.sum((points_final)**2),axis=2)
        LD = np.sum(LDrows, axis=0)
    return LD

Removed `dt` and `p` from functions since these are global variables.

<span style='color:red'>What about flag_m?</span>

In [ ]:
def compute_lagrangian_descriptor(points_initial, vector_field, time_interval, dt, norm = False):
    f = vector_field_flat # issue: defined as global function, independently of vector_field argument
    t_initial, t_final = time_interval
    if (t_final-t_initial)*dt<0:
        dt=-dt
    
    y0 = points_initial.flatten() # all trajectories at once?
    teval = np.arange(t_initial, t_final+dt,dt) #+dt needed to stop at t_final, previous version terminated early
    # Outputs from integration
    solution_object = integrator(f, time_interval, y0, t_eval=teval, args=(vector_field, )) #f and vector_field ambiguity needs fixing
    points_final = solution_object.y.T
    points_final = points_final.reshape((len(teval),shape[0],shape[1]))
    # Compute LD
    LD = lagrangian_descriptor(points_final)
    return LD

Break down `lagrangian_descriptors` into separate code chunks

In [ ]:
def draw_lagrangian_descriptor(LD, flag_m, flag_type):
    LD = LD.reshape(Nx, Ny).T # Reshape 1D array
#     LD = LD / LD.max()  # Scale LD output
    ###################################
    # Plot LDs
    fig,ax = plt.subplots(1,1,dpi=100)
    
    points_x = np.linspace(x_min, x_max, Nx)
    points_y = np.linspace(y_min, y_max, Ny)    
    X, Y = np.meshgrid(points_x, points_y)

    scatter = plt.scatter(X,Y,c=LD,cmap='bone')
    ###################################
    # Customise appearance
    if flag_m:
        str_meth = ' '.join(['p-norm (p=',str(p_norm),')--'])
    else:
        str_meth = 'arclength --'
    
    if flag_type == 1:
        string_title = ['Forward LDs ', str_meth, '(','$\\tau=$ ',str(tau),' , ','$t_0=$',str(t0),')']
    elif flag_type == 2:
        string_title = ['Backward LDs ', str_meth, '(','$\\tau=$ ',str(tau),' , ','$t_0=$',str(t0),')']
    else:
        string_title = ['LDs ', str_meth, '(','$\\tau=$ ',str(tau),' , ','$t_0=$',str(t0),')']
    
    string_title = ' '.join(string_title)
    
    ax.set_title(string_title, fontsize=12)
    ax.set_xlabel('x', fontsize=18)
    ax.set_ylabel('y', fontsize=18)
    ax.set_aspect('auto')

    fig.colorbar(scatter) # Add color bar

    plt.show()

# Benchmark examples

## Vector fields

Define vector field $f(x,y) = (f_1, f_2)$

For for the Initial Value problem 

\begin{align*}
    \dot{x} &= dx/dt = f_1(x,y)\\
    \dot{y} &= dy/dt = f_2(x,y)
\end{align*}

with initial condition $(x_0, y_0) \in \mathbb{R}^2$ at time $t_0 \geq 0$

In [ ]:
def HamCenter1D(t, u):
    x, y = u.T
    # Hamiltonian Model Parameter
    omega = 1
    v = np.array([ omega * y, - omega * x]).T
    return v

In [ ]:
def HamSaddle1D(t, u):
    x, y = u.T
    # Hamiltonian Model Parameter
    lamda = 1
    v = np.array([ lamda * y, lamda * x]).T
    return v

In [ ]:
def forcing(t, u, flag_pert, perturbation_params):
    x, y = u.T
    pert = np.zeros((len(x),1))
    # Perturbation parameters
    A, freq = perturbation_params # Amplitude and Frequency
    if flag_pert == 1:
        perturbation = A * np.sin(freq*t)
    elif flag_pert == 2:
        perturbation = A * np.sech(t) * np.sin(freq*t)
    else:
        perturbation = perturbation
    
    return perturbation

In [ ]:
def HamDuffing1D(t, u):
    x, y = u.T
    # Hamiltonian Model Parameter
    perturbation = forcing(t, u, flag_pert, perturbation_params)
    v = np.array([y, x - x**3 + perturbation]).T
    return v

## Input parameters

In [ ]:
###################################
# Integration parameters
t0 = 0
tau = 12
dt = 0.1
###################################
# Lagrangian descriptor parameters
p_norm = 1/2
flag_m = True
###################################
# Mesh parameters
x_min,x_max = [-1.6, 1.6]
y_min,y_max = [-1, 1]
Nx, Ny = [300, 300]

GRID_PARAMETERS = [(x_min, x_max, Nx), (y_min, y_max, Ny)]

dim = 2
shape = (Nx*Ny, dim)
###################################
# Time Dependent Perurbation (for Hamilton Duffing)
flag_pert = 1;
perturbation_params = [0.15, 0.5]

## Compute and Visualise

### Hamilton Centre

In [ ]:
initial_conditions = generate_points(GRID_PARAMETERS)
vector_field = HamCenter1D

time_interval = (t0, t0 + tau)
LD_forward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_forward, True, 1)

time_interval = (t0, t0 - tau)
LD_backward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_backward, True, 2)

LD_total = LD_forward + LD_backward
draw_lagrangian_descriptor(LD_total, True, 3)

### Hamilton Saddle

In [ ]:
initial_conditions = generate_points(GRID_PARAMETERS)
vector_field = HamSaddle1D

time_interval = (t0, t0 + tau)
LD_forward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_forward, True, 1)

time_interval = (t0, t0 - tau)
LD_backward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_backward, True, 2)

LD_total = LD_forward + LD_backward
draw_lagrangian_descriptor(LD_total, True, 3)

### Hamilton Duffing

In [ ]:
initial_conditions = generate_points(GRID_PARAMETERS)
vector_field = HamDuffing1D

time_interval = (t0, t0 + tau)
LD_forward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_forward, True, 1)

time_interval = (t0, t0 - tau)
LD_backward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_backward, True, 2)

LD_total = LD_forward + LD_backward
draw_lagrangian_descriptor(LD_total, True, 3)

### Hamilton Saddle-Node (IN PROGRESS)

<span style='color:red'><b>TO-DO</b></span> Implement _Variable Time Integration_ for systems like the HSN

In [ ]:
def HamSN1D(t, u):
    x, y = u.T
    # Hamiltonian Model Parameter
    v = np.array([ y, -x -x**2]).T
    return v

In [ ]:
###################################
# Integration parameters
t0 = 0 # Initial time
tau = 1.5 # Time-interval length
dt = 0.1 # Timestep
###################################
# Lagrangian descriptor parameters
p_norm = 1/2 # p-value of Lp norm
flag_m = True
###################################
# Grid parameters
x_min,x_max = [-1.5, 1] # Grid limits X-axis
y_min,y_max = [-1, 1] # Grid limits Y-axis
Nx, Ny = [300, 300] # Number of points per axis

GRID_PARAMETERS = [(x_min, x_max, Nx), (y_min, y_max, Ny)]

dim = 2 # Phase-space Dimension
shape = (Nx*Ny, dim) # Shape of array of grid-points
###################################
# Time Dependent Perurbation (for Hamilton Duffing)
flag_pert = 1 
perturbation_params = [0.15, 0.5] # Amplitude & Frequency, periodic pertubation

In [ ]:
initial_conditions = generate_points(GRID_PARAMETERS)
vector_field = HamSN1D

# Compute and draw forward LD
time_interval = (t0, t0 + tau)
LD_forward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_forward, True, 1)

# Compute and draw backward LD
time_interval = (t0, t0 - tau)
LD_backward = compute_lagrangian_descriptor(initial_conditions, vector_field, time_interval, dt)
draw_lagrangian_descriptor(LD_backward, True, 2)

# Compute and draw total LD
LD_total = LD_forward + LD_backward
draw_lagrangian_descriptor(LD_total, True, 3)